# MVP author search with OpenAlex data source

Stack choice:
- PGVector (Try new thing... not very performant tho.)
- SQLModel (For faster dev, less boilerplate)
- OpenAI (Faster prototyping...)

Steps:
1. Pull from open alex
1. Get abstract and full text (Defer)
1. Get embeddings
1. Push to PGVector


In [ ]:
from openalex_search.db import init
from openalex_search.embedding import embed_work
from openalex_search.db import Work, engine
from sqlalchemy.orm import Session

init(wipe=True)

In [2]:
my_work = Work.pull(doi="10.1016/j.dcn.2023.101292")
my_work = embed_work(my_work)


In [ ]:
with Session(engine) as session:
    session.add(my_work)
    session.commit()